## RAG-application
### Setup och Ladda Databas

In [7]:
import os
from pathlib import Path
from langchain_community.vectorstores import Chroma 
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv

# 1. Definiera sökvägar
BASE_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden")
DB_DIR = BASE_DIR / "data" / "03_vector_db" / "green_power_db"

# 2. Ladda Embedding-modellen (MÅSTE vara samma som i notebook 04!)
print("Laddar embedding-modell (all-mpnet-base-v2)...")
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}, # Vi kör lokalt nu, så CPU är säkrast
    encode_kwargs={'normalize_embeddings': False}
)

# 3. Ladda Vektordatabasen
print(f"Laddar databas från {DB_DIR}...")
if not DB_DIR.exists():
    print(f"VARNING: Mappen {DB_DIR} hittades inte! Kolla sökvägen.")
else:
    vectordb = Chroma(
        persist_directory=str(DB_DIR),
        embedding_function=embedding_model
    )
    print(f"Databas laddad! Innehåller {vectordb._collection.count()} chunks.")

ModuleNotFoundError: No module named 'langchain.chains'

### Testa sökning (Utan LLM)

In [ ]:
# Testa en sökning för att se att retrieval fungerar
query = "Får man bygga solcellsparker på jordbruksmark?"

print(f"Söker efter: '{query}'...")
docs = vectordb.similarity_search(query, k=3) # Hämta topp 3 träffar

print("\n--- Resultat ---")
for i, doc in enumerate(docs):
    print(f"\nDokument {i+1}:")
    print(f"Källa: {doc.metadata.get('source', 'Okänd')}")
    print(f"Textsnutt: {doc.page_content[:300]}...") # Visar första 300 tecknen

### Koppla in Gemini (Chatt)

In [ ]:
# 1. Ladda miljövariabler från .env-filen.
# Detta sätter automatiskt GOOGLE_API_KEY i os.environ
load_dotenv()

# 2. Skapa LLM-objektet (Gemini 1.5 Flash är snabb och billig/gratis)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    temperature=0.3 # Låg temperatur för mer fakta, mindre hittepå
)

# 3. Skapa Prompt-mallen (Din "Persona")
template = """
Du är 'Solaris Legal', en expert på juridik kring solcellsparker i Sverige.
Använd följande kontext (utdrag från domar och ansökningar) för att besvara användarens fråga.

Om svaret inte finns i kontexten, säg "Jag hittar inte svaret i de dokument jag har tillgång till."
Hitta inte på egna fakta utanför kontexten.
Var noga med att hänvisa till källorna (dokumentnamnen) i ditt svar.

Kontext:
{context}

Fråga: {question}

Svar:
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 4. Skapa RAG-kedjan
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 5}), # Skicka med 5 chunks
    return_source_documents=True, # Vi vill ha källorna!
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

print("RAG-bot är redo!")